# Jarduino: Integrating Arduino Development with Jupyter IPython Notebooks

## The Problem - and the Opportunity

Integrating Python and Arduino requires separate environments - the Arduino IDE, and, for Python, a simple editor or an integrated development environment (IDE) like Eclipse or Pycharm. Using a single environment would be nice. For a lot of experimentation, integrating it into the Jupyter Notebook would be even better.

Jupyter Notebook is dominant in data sciences and machine learning because of it's ability to tie together code, notes, data and results. While not as sophisticated as an IDE, it's overall better because of the ability to keep notes, and to manipulate and lot data with tools like numpy, pandas and matplotlib.

Arduino coding consists of (relatively) simpler programs where the Arduino is used as a data collector or as the end point for receiving data/taking action. Complex processing can be passed to a more powerful system for data analysis or to make decisions and pass back an action back to the Arduino. There are Raspberry Pi libraries like __[nanpy](https://nanpy.github.io/)__ to help this integration. The combination of Raspberry Pi and Arduino is popular among robotics developers.

Web searches for Arduino/Jupyter integration show a lot usage of Arduino data inside Jupyter, but with the Arduino code always external. The code may be shown in the notebook, but always with the caveat *'you need to copy this code to the Arduino IDE to load and run on your board'*.

"Why hasn't somebody done something about this?", I thought, while unfortunately standing in front of a mirror.

In this paper, we'll go through the process of how we developed the %%jarduino and %jardutil magics to help manage complex multi-board development of Arduino code. Along the way we'll also show you some tips on how to manage and load Arduino code using only standard Jupyter IPython features.


## Design Considerations

[Jupyter has dozens of language kernels](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels), none of which support Arduino sketches. It's allegedly not too difficult to [create a new kernel for Jupyter](https://jupyter-client.readthedocs.io/en/stable/kernels.html). Arduino sketches are based on C, so using a C kernel, particularly with Jupyter cells supporting the language, could be a starting point.

But a goal of the project is to be able to work on Python and Arduino code together. And so we fall into a serious design issue.

Each Jupyter Notebook supports a single kernel - which means a single language. A Jupyter notebook is composed of cells, the main types being __markdown__ for text and images (you are readin a markdown cell), and __code__ for the programs. Code cells are directly tied to the kernel, so in a Python notebook, code means Python. There is no option for other types of code cells in a Python notebook, or for Python code cells in a notebook for another language.

This is some healthy debate about this Jupyter development circles. The Jupyter designers' view is that supporting multiple languages is best handled with messaging between different kernels. A big proponent of multi-language notebooks is Bo Peng, the developer of the multi-kernel Jupyter variant __[SoS Polyglot Notebook and Workflow](https://vatlab.github.io/sos-docs/)__ system. You can check out some __[to-and-fro on the topic here](https://github.com/jupyterlab/jupyterlab/issues/2815)__, with each side respectfully covering each other's arguments and views. It's a good read and a nice change from the usual Internet flame wars.

But is this complexity necessary for Arduino Sketches?

Sketches, regardless of complexity, are relatively limited programs. They are not calling databases or writing to the web. They are not calling the other Jupyter libraries. They are limited by the microcontroller's memory and processing capabilities.

Developing and running a sketch requires three things:

*  Edit and saving the code
*  Running a verify/compile on the code
*  Loading the resulting program to the Arduino board

If we can accomplish these things and maintain our Python coding ability, we'll meet our goals

## The other way of running non-Python code in an IPython notebook

In Jupyter's Python cells, support for other languages is often provided by a [*magic*](https://ipython.readthedocs.io/en/stable/interactive/magics.html). Magics are macro-like operations written in Python that can perform a utility function independently or using the contents of the cell.

In the example below, the *%%bash* magic takes the rest of the cell as the bash script and runs it.

In [ ]:
%%bash -s "$PATH"
echo "This bash script see that my path is $1"
echo "Isn't that cool?"

It is still a Python cell - but holding bash code.

Other magics allow you to run commands. For example, the %%cmd magic executes the rest of the cell as separate operating system commands.

In [3]:
%%cmd
echo "Listing directory sketches"
dir sketches

Microsoft Windows [Version 10.0.17763.316]
(c) 2018 Microsoft Corporation. All rights reserved.

C:\Users\rjjun\Notebooks\ylab_Jupyter_Intro>echo "Listing directory sketches"
"Listing directory sketches"

C:\Users\rjjun\Notebooks\ylab_Jupyter_Intro>dir sketches
 Volume in drive C is Windows
 Volume Serial Number is 1A72-5394

 Directory of C:\Users\rjjun\Notebooks\ylab_Jupyter_Intro\sketches

2019-02-27  08:17 AM    <DIR>          .
2019-02-27  08:17 AM    <DIR>          ..
2019-02-27  08:17 AM    <DIR>          arduinotest
2019-02-27  08:17 AM    <DIR>          stupid
               0 File(s)              0 bytes
               4 Dir(s)  437,652,611,072 bytes free

C:\Users\rjjun\Notebooks\ylab_Jupyter_Intro>

Fortunately, the __[Arduino IDE has a command-line interface (CLI)](https://github.com/arduino/Arduino/blob/master/build/shared/manpage.adoc)__ supports what we need to compile and load our sketches.

## Proof of concept testing

There are two types of magics in IPython.

*  __%linemagics__ start with one % symbol. These are single-line commands that operate based on parameters on appearing on the same line. You can have multiple %linemagics in a single cell
*  __%%cellmagics__ start with two %% symbols. You can only have one in a cell, and it appears at the top of the cell. The entire cell content is passed to the magic.

Some sites provide lists of magics, but even then the documentation is pretty sparse. The library of magics keeps evolving, so many sites are out of date. After searching a few sites, I found two key features.

__*%lsmagic*__ provides a list of all available magics.

Directly appending a ? to the name of the magic (no space) provides help. __*%%writefile?*__ will provide the command help and syntax.

__%%writefile <filename>_ is a built-in magic, that  dumps everything in the cell - with the exception of the %%writefile line - to the specified file. The next cell is an example - where we just happend to fill the cell with a simple Arduino sketch

In [16]:
%%writefile arduinotest.ino
void setup(){}
void loop(){
    int x = 10;
}

Overwriting arduinotest.ino


So we've written our Arduino code in our notebook and saved it to a file. That's a great starting point.

Now we need to compile and load it.

The __[Arduino IDE command-line interface](https://github.com/arduino/Arduino/blob/master/build/shared/manpage.adoc)__ gives us the two options:

*  __Arduino --verify <filename>__ to run the verify/compile phase. Very handy for testing when we don't have an Arduino board connected.
*  __Arduino --upload <filename>__ to verify/compile and load the code on the board.
    
There are more command line options that we'll get to later.
    
A bit of trial and error shows that the Arduino installation doesn't add the command to the operating system path. We could add it to the operating system PATH, or just run it with a fully-qualified path.

The next cell show the results.

In [13]:
%%cmd
"C:\Program Files (x86)\Arduino\Arduino" --upload arduinotest.ino

Microsoft Windows [Version 10.0.17763.316]
(c) 2018 Microsoft Corporation. All rights reserved.

C:\Users\rjjun\Notebooks\ylab_Jupyter_Intro>"C:\Program Files (x86)\Arduino\Arduino" --verify arduinotest.ino

C:\Users\rjjun\Notebooks\ylab_Jupyter_Intro>Sketch uses 444 bytes (1%) of program storage space. Maximum is 32256 bytes.
Global variables use 9 bytes (0%) of dynamic memory, leaving 2039 bytes for local variables. Maximum is 2048 bytes.


Loading configuration...
Initializing packages...
Preparing boards...
Verifying...


Success! With two simple magics, we wrote, compiled and loaded our Arduino code without leaving the notebook.

The %%cmd example above will work with any existing file.

But what if we want to load and edit an existing file?

In [ ]:
%load arduinotest.ino

will load our code into a cell, with the result below.

In [ ]:
# %load arduinotest.ino
void setup(){}
void loop(){
    int x = 10;
}


## Streamlining and enhancing the process


Next step is doing it from a single cell - not just the %%writefile, but also the compilation and upload, with some checking of results along the way. But remember we can only have one cell magic in each cell. We can't run %%writefile and %% cmd from the same cell.

Writing our own magic should do the trick. Combine the capabilities of %%write and %cmd. We can write Python scripts to wrap around the Arduino commands and manage the output messages and return codes. 

We found web sites on __[creating a magic](http://mlexplained.com/2017/12/28/creating-custom-magic-commands-in-jupyter/)__ and __[another one](https://stackoverflow.com/questions/38020679/jupyter-write-a-custom-magic-that-modifies-the-contents-of-the-cell-its-in)__ and possibly __[the best one](https://ipython-books.github.io/14-creating-an-ipython-extension-with-custom-magic-commands/)__. If you haven't used some of the fancier features of Python like function decorators, it can appear pretty obtuse. The example are described, but not commented.

> Information on function decorators __[here (tutorial)](https://www.python-course.eu/python3_decorators.php)__ , __[here](https://www.thecodeship.com/patterns/guide-to-python-function-decorators/)__ and __[here (more theoretical)](https://www.thecodeship.com/patterns/guide-to-python-function-decorators/)__.

> The function decorators used in the examples are based on the [Python argparse library](https://docs.python.org/3/howto/argparse.html). If you haven't used it, read up on it before attempting to write or modify a magic.



## Jarduino magics: %%jarduino and %jardutil

The Jarduino magics can be found in file *jarduino_magic.py* located [here](https://github.com/ylabrj/Jarduino).  [README.md](https://github.com/ylabrj/Jarduino/blob/master/README.md) file has the latest information on installation and usage issues.

 The functions are available through two magics:
 
 *  __%%jarduino__ is a Python cell magic where the rest of the cell is the Arduino code (instead of Python code!)
 * __%jardutil__ is a Python line magic that operates on existing sketch files and provides utility functions and graphing extensions.
 
 We'll take a walk-through to show how the features are used and how they evolved in our test and development environment.
 
### %%jarduino  <filename>
    
This is the cell magic, so it expects the cell to contain the arduino code.

In it's most basic usage, it saves and compiles/loads the code onto a connected Arduino board.

In [5]:
%%jarduino mysketch
#define NUMBER 10
int x;
void setup(){
    Serial.begin(9600);
    x = NUMBER;
}
void loop(){
    Serial.println(x);
    x++;
}

Overwriting C:\Users\rjjun\Notebooks\ylab_Jupyter_Intro\sketches\mysketch\mysketch.ino
-- check option: Compile only - will not attempt to load to board
Starting Arduino build
Command:  Arduino --verify C:\Users\rjjun\Notebooks\ylab_Jupyter_Intro\sketches\mysketch\mysketch.ino
Command output :  Sketch uses 1758 bytes (5%) of program storage space. Maximum is 32256 bytes.
Global variables use 190 bytes (9%) of dynamic memory, leaving 1858 bytes for local variables. Maximum is 2048 bytes.

Command errors:  Loading configuration...
Initializing packages...
Preparing boards...
Verifying...

Done


In addition to compiling, the sketch did several things when saving the file.

* It saves the cell contents (less the %%jarduino line) in file *sketches/mysketch/mysketch.ino

* To keep things tidy, %%jarduino places all the sketch directories under directory *sketches/* in the current notebook directory. If it doesn't exist, *sketches/* is automatically created.
* Following the convention of the Arduino IDE, the sketch is created in a directory with the same name as the file. That directory is also created automatically.

> __*This file layout is critical for the Arduino IDE*__. When compiling from within a directory, the Arduino IDE appends together *all of the files in a directory*. If you have two files that each have *void setup()* and *void loop()* functions, the compile will crash because of the duplicate function definitions. In the Arduino IDE, separate files used as function libraries within the same directory must be loaded and managed as separate tabs in the IDE.

The filename can be specified as *mysketch* or *mysketch.ino*. Both will be handled properly, using directory *sketches/mysketch/*

### --dir: Overriding the default directory
If you want to override the directory in a manner similar to the Arduino tabs, you can write your function with the --dir option as shown in the following two examples

In [ ]:
%%jarduino myfunction --dir mysketch
void myfunction(){
    Serial.println("Hey! I wrote a function!");
}

The Arduino IDE running the compile load behaves as expected, compiling together mysketch.ino and myfunction.ino as well as any other .ino found in the directory.

To avoid the default *sketches/* subdirectory, use an absolute path name starting with / or, in Windows, DRIVE:/

### --verify: compile only

If you don't have an Arduino board attached to your system, you can run a compile without loading to the board with
> %%jarduino mysketch --verify

### --port -p and --board: Arduino serial port and board type

The Arduino IDE will normally identify the Arduino port and device type. Because multiple Arduino boards and types can be connected, you can override the default with either or both of --port and --board

> %%jarduino mysketch --board micro --port COM6

### --verbose and --quiet
*  --verbose option is passed to the Arduino IDE, causing it to vomit up an astounding amount of status and diagnostic messages.
*  --quiet eliminates the default Arduino IDE messages that are sent to stdout and stderr. This is strictly under the control of the jarduino magic and does not override --verbose, which is passed to the IDE.

### --redefine: parameter passing to change #define statements

If a sketch contains #define statements such as

> #define BLINKPIN 13

--redefine can be used to change the value without going changing the cell. It will change the stored file

In the following example, the file will be saved with *#define BLINKPIN 11* before compiling. The cell remains as is.


In [ ]:
%%jarduino blinksketch --redefine BLINKPIN 11
#define BLINKPIN 13
#define SERIALSPEED 9600
void setup(){
    
}

Multiple --redefines are allowed on the command line. For example:

> %%jarduino blinksketch --redefine BLINKPIN 11 --redefine SERIALSPEED 19200


The feature is particularly useful when dealing with multiple boards and types when pins may be laid out differently. A powerful example is provided in the *%jardutil* description.

> __*WARNING*: YES, THIS CHANGES THE FILE CONTENT AND DOES NOT RESTORE IT AFTER__


In [ ]:
%jardutil?

### %jardutil line magic
%jardutil
